In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
num_classes = 1000
n_preds = 200
input_dir = os.path.join(os.getcwd(), '../outputs')
df_output = pd.read_csv('sample.csv')
df_output = df_output.set_index('Class')
df_inputs = pd.DataFrame(columns=['vid_id', 'vid_label_pred', 'seg_time_pred'])

In [ ]:
df_vocab = pd.read_csv('../data/3rd_challenge_vocabulary.csv')
vocab_idx2label_dict = dict()
for i, label in enumerate(df_vocab['Index']):
    vocab_idx2label_dict[i+1] = label

In [ ]:
n_exist_files = 0
for i in range(1, num_classes+1):
    if i % 100 == 0:
        print(i)
    df_input = pd.read_csv(os.path.join(input_dir, '%04d.csv'%(i)))
    if len(df_input) == 0:
        continue
    df_input = df_input.sort_values(by=['vid_prob'], ascending=False).head(n_preds)
    df_input = df_input[:n_preds]
    df_input = df_input.drop(columns=['vid_prob'])
    df_input['vid_label_pred'] = df_input['vid_label_pred'].map(vocab_idx2label_dict)
    df_input['seg_time_pred'] = 1
    df_inputs = pd.concat([df_inputs, df_input], axis=0)
    n_exist_files += 1
print("n_exist_files:", n_exist_files)

In [ ]:
df_inputs

In [ ]:
df_inputs = pd.concat([
    pd.DataFrame(
        {'vid_id': row.vid_id,
         'vid_label_pred': row.vid_label_pred,
         'seg_time_pred': np.arange(30, 270, 5)
        }
    ) for i, row in df_inputs.iterrows()
], ignore_index=True).loc[:, ['vid_id', 'vid_label_pred', 'seg_time_pred']]
df_inputs

In [ ]:
df_output = df_output.join(df_inputs.set_index('vid_label_pred'), on='Class')
df_output

In [ ]:
df_output['vid_id'] = df_output['vid_id'].fillna('ABCD')
df_output['seg_time_pred'] = df_output['seg_time_pred'].fillna(0)
df_output

In [ ]:
df_output['seg_time_pred'] = df_output['seg_time_pred'].astype(int)

In [ ]:
df_output['Segments'] = df_output[['vid_id', 'seg_time_pred']].apply(lambda x: ':'.join(map(str, x)), axis=1)

In [ ]:
df_output

In [ ]:
df_output = df_output.groupby('Class').agg({'Segments': lambda x: ' '.join(map(str, x))})

In [ ]:
df_output.to_csv("sample_submission.csv")

In [ ]:
df_output